In [12]:
import pyodbc

In [13]:
cnxn_str = (
    """Driver=SQL Server Native Client 11.0;
    Server=.;
    Database=Dormitory;
    Trusted_Connection=Yes"""
)

cnxn = pyodbc.connect(cnxn_str)

In [14]:
cursor = cnxn.cursor()

In [30]:
#список комнат и количество студентов в каждой из них
result = cursor.execute(
    """SELECT r.name, 
    count(s.id) as count_students
FROM [dbo].[Rooms] as r 
	LEFT JOIN [dbo].[Students] as s
	ON r.id=s.room
GROUP BY r.name
ORDER BY count(s.id)"""
)

In [31]:
for record in result:
    print(record)

('Room #482', 0)
('Room #422', 2)
('Room #810', 2)
('Room #661', 3)
('Room #677', 3)
('Room #913', 3)
('Room #985', 3)
('Room #419', 3)
('Room #49', 3)
('Room #154', 3)
('Room #157', 4)
('Room #134', 4)
('Room #178', 4)
('Room #25', 4)
('Room #345', 4)
('Room #353', 4)
('Room #498', 4)
('Room #6', 4)
('Room #611', 4)
('Room #443', 4)
('Room #484', 4)
('Room #487', 4)
('Room #972', 4)
('Room #934', 4)
('Room #681', 4)
('Room #710', 4)
('Room #873', 4)
('Room #811', 5)
('Room #788', 5)
('Room #794', 5)
('Room #798', 5)
('Room #800', 5)
('Room #754', 5)
('Room #772', 5)
('Room #777', 5)
('Room #715', 5)
('Room #698', 5)
('Room #664', 5)
('Room #628', 5)
('Room #636', 5)
('Room #919', 5)
('Room #92', 5)
('Room #943', 5)
('Room #915', 5)
('Room #902', 5)
('Room #97', 5)
('Room #964', 5)
('Room #990', 5)
('Room #466', 5)
('Room #436', 5)
('Room #396', 5)
('Room #382', 5)
('Room #383', 5)
('Room #571', 5)
('Room #522', 5)
('Room #538', 5)
('Room #540', 5)
('Room #333', 5)
('Room #3', 5)
('Roo

In [22]:
for record in cursor:
    print(record)


('Room #482', 0)
('Room #422', 2)
('Room #810', 2)
('Room #661', 3)
('Room #677', 3)
('Room #913', 3)
('Room #985', 3)
('Room #419', 3)
('Room #49', 3)
('Room #154', 3)
('Room #157', 4)
('Room #134', 4)
('Room #178', 4)
('Room #25', 4)
('Room #345', 4)
('Room #353', 4)
('Room #498', 4)
('Room #6', 4)
('Room #611', 4)
('Room #443', 4)
('Room #484', 4)
('Room #487', 4)
('Room #972', 4)
('Room #934', 4)
('Room #681', 4)
('Room #710', 4)
('Room #873', 4)
('Room #811', 5)
('Room #788', 5)
('Room #794', 5)
('Room #798', 5)
('Room #800', 5)
('Room #754', 5)
('Room #772', 5)
('Room #777', 5)
('Room #715', 5)
('Room #698', 5)
('Room #664', 5)
('Room #628', 5)
('Room #636', 5)
('Room #919', 5)
('Room #92', 5)
('Room #943', 5)
('Room #915', 5)
('Room #902', 5)
('Room #97', 5)
('Room #964', 5)
('Room #990', 5)
('Room #466', 5)
('Room #436', 5)
('Room #396', 5)
('Room #382', 5)
('Room #383', 5)
('Room #571', 5)
('Room #522', 5)
('Room #538', 5)
('Room #540', 5)
('Room #333', 5)
('Room #3', 5)
('Roo

In [23]:
import pandas

In [24]:
#top 5 комнат, где самый маленький средний возраст студентов
cursor.execute(
    """DECLARE @Now  date
SET @Now=GETDATE()


SELECT name, avg_age
	   
FROM (

		SELECT TOP 5 r.name,
		       AVG(CONVERT(int,CONVERT(char(8),@Now,112))-CONVERT(char(8),s.[birthday],112))/10000 as avg_age,
			   DENSE_RANK() OVER (PARTITION BY r.name 
				ORDER BY (AVG(CONVERT(int,CONVERT(char(8),@Now,112))-CONVERT(char(8),s.[birthday],112))/10000)) as rank
	    FROM [dbo].[Rooms] as r 
			LEFT JOIN [dbo].[Students] as s
			ON r.id=s.room
		WHERE s.[birthday] IS NOT NULL
		GROUP BY r.name
		ORDER BY  AVG(CONVERT(int,CONVERT(char(8),@Now,112))-CONVERT(char(8),s.[birthday],112))/10000
         ) t

WHERE rank <= 5 """
)

In [25]:
for record in cursor:
    print(record)

('Room #661', 9)
('Room #913', 17)
('Room #111', 29)
('Room #957', 30)
('Room #773', 32)


In [26]:
#top 5 комнат с самой большой разницей в возрасте студентов
cursor.execute(
    """DECLARE @Now  date
SET @Now=GETDATE()

SELECT [room], difference

FROM (
	SELECT TOP 5 [room],
		(MAX(CONVERT(int,CONVERT(char(8),@Now,112))-CONVERT(char(8),[birthday],112))/10000) -
		(MIN(CONVERT(int,CONVERT(char(8),@Now,112))-CONVERT(char(8),[birthday],112))/10000) as difference,
		DENSE_RANK () OVER (ORDER BY ((MAX(CONVERT(int,CONVERT(char(8),@Now,112))-CONVERT(char(8),[birthday],112))/10000) -
									  (MIN(CONVERT(int,CONVERT(char(8),@Now,112))-CONVERT(char(8),[birthday],112))/10000)) DESC) as  max_difference
	FROM [dbo].[Students]
	GROUP BY [room]
	) t

WHERE max_difference <= 5"""
)

In [27]:
for record in cursor:
    print(record)

(381, 116)
(905, 115)
(213, 115)
(712, 115)
(875, 115)


In [28]:
#список комнат где живут разнополые студенты
cursor.execute(
    """SELECT DISTINCT room
FROM (
	SELECT room,
	       sex,
           count(sex) as count_sex,
		   rank() over (partition by room order by sex desc) as rank
	FROM [dbo].[Students] as s 
	GROUP BY room, sex
	) t
WHERE rank=2"""
)

In [29]:
for record in cursor:
    print(record)

(0, )
(1, )
(2, )
(3, )
(4, )
(5, )
(7, )
(8, )
(9, )
(10, )
(11, )
(12, )
(13, )
(14, )
(15, )
(16, )
(17, )
(18, )
(19, )
(20, )
(21, )
(22, )
(23, )
(24, )
(25, )
(26, )
(27, )
(28, )
(29, )
(30, )
(31, )
(32, )
(33, )
(34, )
(35, )
(36, )
(37, )
(38, )
(39, )
(40, )
(41, )
(42, )
(43, )
(44, )
(45, )
(46, )
(47, )
(48, )
(49, )
(50, )
(51, )
(52, )
(53, )
(54, )
(55, )
(56, )
(57, )
(58, )
(59, )
(60, )
(61, )
(62, )
(63, )
(64, )
(65, )
(66, )
(67, )
(68, )
(69, )
(70, )
(71, )
(72, )
(73, )
(74, )
(75, )
(76, )
(77, )
(78, )
(79, )
(80, )
(81, )
(82, )
(83, )
(84, )
(85, )
(86, )
(87, )
(88, )
(89, )
(90, )
(91, )
(92, )
(93, )
(94, )
(95, )
(96, )
(97, )
(98, )
(99, )
(100, )
(101, )
(102, )
(103, )
(104, )
(105, )
(106, )
(107, )
(108, )
(109, )
(110, )
(111, )
(112, )
(113, )
(114, )
(115, )
(116, )
(117, )
(118, )
(119, )
(120, )
(121, )
(122, )
(123, )
(124, )
(125, )
(126, )
(127, )
(128, )
(129, )
(130, )
(131, )
(132, )
(133, )
(134, )
(135, )
(136, )
(137, )
(138, )
(139

In [19]:
cnxn.commit()